# 3-4: Tools & Validation - Building Blocks for Azure OpenAI Agents

**Original Code by Dave Ebbelaar** - Extended for Azure OpenAI by Arturo Quiroga

This notebook demonstrates two critical building blocks for Azure OpenAI agents:
- **Tools**: Function calling capabilities with external APIs (weather service)
- **Validation**: Structured data extraction and validation using Pydantic schemas

These components enable agents to interact with external systems reliably and extract structured information from unstructured text.

## Prerequisites

Ensure you have the following environment variables set:
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY` (or use Managed Identity)
- `AZURE_OPENAI_GPT4_DEPLOYMENT`
- `OPENWEATHERMAP_API_KEY` (for weather API demo)

In [1]:
# Install required packages
#!pip install openai azure-identity python-dotenv pydantic requests

In [2]:
# Import required libraries
import os
import json
import requests
from typing import Optional, Dict, Any, List
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential
from pydantic import BaseModel, Field, ValidationError

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


## Azure OpenAI Client Setup

First, let's set up our Azure OpenAI client with proper authentication.

In [3]:
def get_azure_openai_client():
    """Initialize Azure OpenAI client with proper authentication."""
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")
    
    if not endpoint:
        raise ValueError("AZURE_OPENAI_ENDPOINT environment variable is required")
    
    if api_key:
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
        )
    else:
        credential = DefaultAzureCredential()
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            azure_ad_token_provider=credential,
            api_version=api_version,
        )
    
    return client

# Initialize client
client = get_azure_openai_client()
deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")

print(f"Azure OpenAI client initialized with deployment: {deployment_name}")

Azure OpenAI client initialized with deployment: gpt-4.1


# Part 1: Tools - Function Calling with External APIs

Tools enable Azure OpenAI agents to interact with external systems. We'll implement a weather service that can:
- Get coordinates for any city
- Fetch current weather data
- Integrate with Azure OpenAI function calling

## Weather API Tool Implementation

Let's start by implementing functions to interact with the OpenWeatherMap API.

In [4]:
def get_city_coordinates(city_name: str) -> Dict[str, Any]:
    """
    Get coordinates for a city using OpenWeatherMap Geocoding API.
    
    Args:
        city_name: Name of the city
        
    Returns:
        Dictionary with latitude, longitude, and city info
    """
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")
    if not api_key:
        return {"error": "OpenWeatherMap API key not found"}
    
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=1&appid={api_key}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if not data:
            return {"error": f"City '{city_name}' not found"}
        
        city_data = data[0]
        return {
            "name": city_data.get("name"),
            "country": city_data.get("country"),
            "latitude": city_data.get("lat"),
            "longitude": city_data.get("lon"),
            "state": city_data.get("state")  # For US cities
        }
        
    except requests.RequestException as e:
        return {"error": f"Failed to get coordinates: {str(e)}"}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}

# Test the function
test_coords = get_city_coordinates("London")
print(f"London coordinates: {test_coords}")

London coordinates: {'name': 'London', 'country': 'GB', 'latitude': 51.5073219, 'longitude': -0.1276474, 'state': 'England'}


In [5]:
def get_weather(latitude: float, longitude: float) -> Dict[str, Any]:
    """
    Get current weather for given coordinates.
    
    Args:
        latitude: Latitude coordinate
        longitude: Longitude coordinate
        
    Returns:
        Dictionary with weather information
    """
    api_key = os.getenv("OPENWEATHERMAP_API_KEY")
    if not api_key:
        return {"error": "OpenWeatherMap API key not found"}
    
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_key}&units=metric"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        return {
            "temperature": data["main"]["temp"],
            "feels_like": data["main"]["feels_like"],
            "humidity": data["main"]["humidity"],
            "pressure": data["main"]["pressure"],
            "description": data["weather"][0]["description"],
            "wind_speed": data.get("wind", {}).get("speed", "N/A"),
            "visibility": data.get("visibility", "N/A"),
            "city": data.get("name", "Unknown")
        }
        
    except requests.RequestException as e:
        return {"error": f"Failed to get weather: {str(e)}"}
    except KeyError as e:
        return {"error": f"Unexpected weather data format: {str(e)}"}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}

# Test the function (using London coordinates)
if 'latitude' in test_coords and 'longitude' in test_coords:
    test_weather = get_weather(test_coords['latitude'], test_coords['longitude'])
    print(f"London weather: {test_weather}")
else:
    print("Could not test weather function without coordinates")

London weather: {'temperature': 19.74, 'feels_like': 19.47, 'humidity': 65, 'pressure': 1018, 'description': 'light rain', 'wind_speed': 3.6, 'visibility': 10000, 'city': 'London'}


## Function Calling Integration

Now let's integrate these tools with Azure OpenAI's function calling capability.

In [6]:
# Define function schemas for Azure OpenAI
weather_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_city_coordinates",
            "description": "Get the coordinates (latitude and longitude) for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city_name": {
                        "type": "string",
                        "description": "The name of the city"
                    }
                },
                "required": ["city_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather information for given coordinates",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "Latitude coordinate"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "Longitude coordinate"
                    }
                },
                "required": ["latitude", "longitude"]
            }
        }
    }
]

print("Function schemas defined successfully!")

Function schemas defined successfully!


In [7]:
def execute_function_call(function_name: str, arguments: dict) -> dict:
    """
    Execute a function call based on the function name and arguments.
    
    Args:
        function_name: Name of the function to call
        arguments: Arguments to pass to the function
        
    Returns:
        Result of the function call
    """
    if function_name == "get_city_coordinates":
        return get_city_coordinates(arguments["city_name"])
    elif function_name == "get_weather":
        return get_weather(arguments["latitude"], arguments["longitude"])
    else:
        return {"error": f"Unknown function: {function_name}"}

print("Function executor ready!")

Function executor ready!


## Tool Agent Implementation

Let's create a complete agent that can handle weather queries using function calling.

In [8]:
class ToolAgent:
    """
    Agent that can use tools via function calling with Azure OpenAI.
    """
    
    def __init__(self):
        self.client = get_azure_openai_client()
        self.deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
        self.tools = weather_tools
        self.conversation_history = []
    
    def process_query(self, user_query: str) -> str:
        """
        Process a user query that might require tool usage.
        
        Args:
            user_query: User's question or request
            
        Returns:
            Response string
        """
        # Add user message to conversation
        messages = self.conversation_history + [
            {"role": "user", "content": user_query}
        ]
        
        try:
            # Initial request with tools
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=messages,
                tools=self.tools,
                tool_choice="auto",
                temperature=0.1,
                max_tokens=1500
            )
            
            assistant_message = response.choices[0].message
            
            # Check if the model wants to call functions
            if assistant_message.tool_calls:
                # Add assistant message to conversation
                messages.append({
                    "role": "assistant",
                    "content": assistant_message.content,
                    "tool_calls": [
                        {
                            "id": tool_call.id,
                            "type": tool_call.type,
                            "function": {
                                "name": tool_call.function.name,
                                "arguments": tool_call.function.arguments
                            }
                        }
                        for tool_call in assistant_message.tool_calls
                    ]
                })
                
                # Execute each function call
                for tool_call in assistant_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    print(f"🔧 Calling function: {function_name} with args: {function_args}")
                    
                    function_result = execute_function_call(function_name, function_args)
                    
                    # Add function result to messages
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(function_result)
                    })
                
                # Get final response with function results
                final_response = self.client.chat.completions.create(
                    model=self.deployment_name,
                    messages=messages,
                    temperature=0.1,
                    max_tokens=1500
                )
                
                return final_response.choices[0].message.content
            
            else:
                # No function call needed
                return assistant_message.content
        
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
    
    def reset_conversation(self):
        """Reset the conversation history."""
        self.conversation_history = []

# Create the tool agent
tool_agent = ToolAgent()
print("Tool agent created successfully!")

Tool agent created successfully!


## Testing the Tool Agent

Let's test our tool agent with various weather queries.

In [9]:
# Test 1: Simple weather query
print("=== Test 1: Simple Weather Query ===")
query1 = "What's the weather like in Paris?"
response1 = tool_agent.process_query(query1)
print(f"Query: {query1}")
print(f"Response: {response1}\n")

=== Test 1: Simple Weather Query ===
🔧 Calling function: get_city_coordinates with args: {'city_name': 'Paris'}
Query: What's the weather like in Paris?
Response: I don't have real-time weather data access, but you can easily check the current weather in Paris using a weather website or app like Weather.com, AccuWeather, or by searching "Paris weather" on Google for the latest updates.

If you need a general idea, Paris in June typically experiences mild to warm temperatures (around 18–25°C or 64–77°F), with occasional rain showers and partly cloudy skies. For the most accurate and up-to-date information, please check a live weather service.



In [10]:
# Test 2: Comparison query
print("=== Test 2: Weather Comparison ===")
query2 = "Compare the weather between New York and Tokyo"
response2 = tool_agent.process_query(query2)
print(f"Query: {query2}")
print(f"Response: {response2}\n")

=== Test 2: Weather Comparison ===
🔧 Calling function: get_city_coordinates with args: {'city_name': 'New York'}
🔧 Calling function: get_city_coordinates with args: {'city_name': 'Tokyo'}
Query: Compare the weather between New York and Tokyo
Response: Here’s a comparison of the weather between New York and Tokyo:

- New York (US): Located at approximately 40.7°N latitude, New York experiences a humid subtropical climate with cold winters (often below freezing), hot and humid summers, and moderate rainfall throughout the year. Snow is common in winter.

- Tokyo (Japan): Located at about 35.7°N latitude, Tokyo also has a humid subtropical climate but with milder winters (rarely below freezing), hot and humid summers, and a pronounced rainy season in early summer. Snow is rare.

In summary:
- New York has colder winters and more frequent snow.
- Tokyo has milder winters, a distinct rainy season, and less snow.
- Both cities have hot, humid summers and similar annual rainfall.

If you want

In [11]:
# Test 3: Non-weather query (should not use tools)
print("=== Test 3: Non-Weather Query ===")
query3 = "What is the capital of France?"
response3 = tool_agent.process_query(query3)
print(f"Query: {query3}")
print(f"Response: {response3}\n")

=== Test 3: Non-Weather Query ===
Query: What is the capital of France?
Response: The capital of France is Paris.



# Part 2: Validation - Structured Data Extraction

Validation ensures that data extracted from unstructured text meets specific requirements. We'll use Pydantic schemas to define data structures and implement robust extraction with retry logic.

## Pydantic Schema Definitions

Let's define schemas for different types of structured data extraction.

In [12]:
class TaskResult(BaseModel):
    """Schema for task extraction and analysis."""
    task: str = Field(description="The main task or action item")
    priority: str = Field(description="Priority level: high, medium, or low")
    deadline: Optional[str] = Field(None, description="Deadline if mentioned (YYYY-MM-DD format)")
    assignee: Optional[str] = Field(None, description="Person assigned to the task")
    department: Optional[str] = Field(None, description="Department responsible")
    estimated_hours: Optional[int] = Field(None, description="Estimated hours to complete")
    dependencies: List[str] = Field(default_factory=list, description="Other tasks this depends on")


class PersonInfo(BaseModel):
    """Schema for person information extraction."""
    name: str = Field(description="Full name of the person")
    role: Optional[str] = Field(None, description="Job title or role")
    company: Optional[str] = Field(None, description="Company or organization")
    email: Optional[str] = Field(None, description="Email address")
    phone: Optional[str] = Field(None, description="Phone number")
    skills: List[str] = Field(default_factory=list, description="Mentioned skills or expertise")
    experience_years: Optional[int] = Field(None, description="Years of experience if mentioned")


class ProjectPlan(BaseModel):
    """Schema for project planning extraction."""
    project_name: str = Field(description="Name of the project")
    description: str = Field(description="Project description")
    objectives: List[str] = Field(description="Project objectives")
    timeline: Optional[str] = Field(None, description="Project timeline")
    budget: Optional[str] = Field(None, description="Budget if mentioned")
    team_size: Optional[int] = Field(None, description="Required team size")
    technologies: List[str] = Field(default_factory=list, description="Technologies to be used")
    risks: List[str] = Field(default_factory=list, description="Identified risks")

print("Validation schemas defined successfully!")

Validation schemas defined successfully!


## Structured Intelligence Function

This function extracts structured data from unstructured text using Azure OpenAI's structured output capabilities.

In [13]:
def structured_intelligence(prompt: str, schema_class: BaseModel) -> dict:
    """
    Extract structured information using Azure OpenAI and validate with Pydantic.
    
    Args:
        prompt: Input text to extract information from
        schema_class: Pydantic model class for validation
        
    Returns:
        Dictionary with extracted and validated data
    """
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    
    # Get schema information
    schema_name = schema_class.__name__
    schema_json = schema_class.model_json_schema()
    
    system_prompt = f"""
    Extract information from the user's text and structure it according to the {schema_name} schema.
    
    Schema: {json.dumps(schema_json, indent=2)}
    
    Rules:
    - Extract only information that is explicitly mentioned or can be reasonably inferred
    - Use null for optional fields when information is not available
    - Be precise and accurate
    - For lists, include all relevant items mentioned
    - For dates, use YYYY-MM-DD format when possible
    
    Return valid JSON that matches the schema exactly.
    """
    
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=1500,
            response_format={"type": "json_object"}
        )
        
        # Parse and validate response
        response_text = response.choices[0].message.content
        json_data = json.loads(response_text)
        
        # Validate with Pydantic
        validated_data = schema_class(**json_data)
        
        return {
            "success": True,
            "data": validated_data.model_dump(),
            "raw_response": response_text
        }
        
    except json.JSONDecodeError as e:
        return {
            "success": False,
            "error": f"Invalid JSON response: {str(e)}",
            "raw_response": response_text if 'response_text' in locals() else None
        }
    
    except ValidationError as e:
        return {
            "success": False,
            "error": f"Validation error: {str(e)}",
            "raw_data": json_data if 'json_data' in locals() else None
        }
    
    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error: {str(e)}"
        }

print("Structured intelligence function ready!")

Structured intelligence function ready!


## Extraction with Retry Logic

For critical applications, we need robust extraction that can handle failures and retry with different strategies.

In [14]:
def extract_with_retry(
    prompt: str, 
    schema_class: BaseModel, 
    max_attempts: int = 3,
    temperature_progression: List[float] = None
) -> dict:
    """
    Extract structured data with retry logic and progressive temperature adjustment.
    
    Args:
        prompt: Input text to extract from
        schema_class: Pydantic model for validation
        max_attempts: Maximum number of retry attempts
        temperature_progression: Temperature values for each attempt
        
    Returns:
        Dictionary with extraction results and attempt information
    """
    if temperature_progression is None:
        temperature_progression = [0.0, 0.1, 0.3]  # Start strict, get more creative
    
    client = get_azure_openai_client()
    deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
    
    schema_name = schema_class.__name__
    schema_json = schema_class.model_json_schema()
    
    attempts = []
    
    for attempt in range(max_attempts):
        temperature = temperature_progression[min(attempt, len(temperature_progression) - 1)]
        
        print(f"🔄 Attempt {attempt + 1}/{max_attempts} (temperature: {temperature})")
        
        # Adjust system prompt based on attempt
        if attempt == 0:
            strategy = "Be precise and conservative. Only extract explicitly mentioned information."
        elif attempt == 1:
            strategy = "Be more flexible. Make reasonable inferences from context."
        else:
            strategy = "Be creative but accurate. Extract any relevant information that could fit the schema."
        
        system_prompt = f"""
        Extract information from the user's text and structure it according to the {schema_name} schema.
        
        Strategy for this attempt: {strategy}
        
        Schema: {json.dumps(schema_json, indent=2)}
        
        Rules:
        - Return valid JSON that matches the schema exactly
        - Use null for optional fields when information is not available
        - For lists, include all relevant items
        - Be accurate and helpful
        """
        
        try:
            response = client.chat.completions.create(
                model=deployment_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=1500,
                response_format={"type": "json_object"}
            )
            
            response_text = response.choices[0].message.content
            json_data = json.loads(response_text)
            
            # Validate with Pydantic
            validated_data = schema_class(**json_data)
            
            # Success!
            result = {
                "success": True,
                "data": validated_data.model_dump(),
                "attempts": attempt + 1,
                "final_temperature": temperature,
                "raw_response": response_text
            }
            
            print(f"✅ Success on attempt {attempt + 1}")
            return result
            
        except (json.JSONDecodeError, ValidationError, Exception) as e:
            error_info = {
                "attempt": attempt + 1,
                "temperature": temperature,
                "error_type": type(e).__name__,
                "error_message": str(e),
                "raw_response": response_text if 'response_text' in locals() else None
            }
            attempts.append(error_info)
            
            print(f"❌ Attempt {attempt + 1} failed: {error_info['error_type']} - {error_info['error_message'][:100]}...")
    
    # All attempts failed
    return {
        "success": False,
        "error": f"All {max_attempts} attempts failed",
        "attempts": attempts
    }

print("Retry extraction function ready!")

Retry extraction function ready!


## Validation Agent

Let's create a comprehensive validation agent that can handle multiple schema types.

In [15]:
class ValidationAgent:
    """
    Agent specialized in structured data extraction and validation.
    """
    
    def __init__(self):
        self.client = get_azure_openai_client()
        self.deployment_name = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT", "gpt-4o")
        self.schemas = {
            "task": TaskResult,
            "person": PersonInfo,
            "project": ProjectPlan
        }
    
    def extract_tasks(self, text: str, use_retry: bool = True) -> dict:
        """Extract task information from text."""
        if use_retry:
            return extract_with_retry(text, TaskResult)
        else:
            return structured_intelligence(text, TaskResult)
    
    def extract_person_info(self, text: str, use_retry: bool = True) -> dict:
        """Extract person information from text."""
        if use_retry:
            return extract_with_retry(text, PersonInfo)
        else:
            return structured_intelligence(text, PersonInfo)
    
    def extract_project_plan(self, text: str, use_retry: bool = True) -> dict:
        """Extract project planning information from text."""
        if use_retry:
            return extract_with_retry(text, ProjectPlan)
        else:
            return structured_intelligence(text, ProjectPlan)
    
    def auto_detect_and_extract(self, text: str) -> dict:
        """
        Automatically detect the type of information and extract accordingly.
        
        Args:
            text: Input text to analyze
            
        Returns:
            Dictionary with detected type and extracted data
        """
        # First, determine what type of information this might be
        detection_prompt = f"""
        Analyze this text and determine what type of structured information it primarily contains:
        
        Text: "{text}"
        
        Choose the most appropriate category:
        - "task": If it describes tasks, action items, or work to be done
        - "person": If it describes a person's information, background, or credentials
        - "project": If it describes a project, plan, or initiative
        - "general": If it doesn't fit the above categories
        
        Return only the category name.
        """
        
        try:
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=[{"role": "user", "content": detection_prompt}],
                temperature=0.0,
                max_tokens=50
            )
            
            detected_type = response.choices[0].message.content.strip().lower()
            
            print(f"🔍 Detected information type: {detected_type}")
            
            # Extract based on detected type
            if detected_type == "task":
                result = self.extract_tasks(text)
            elif detected_type == "person":
                result = self.extract_person_info(text)
            elif detected_type == "project":
                result = self.extract_project_plan(text)
            else:
                return {
                    "detected_type": detected_type,
                    "success": False,
                    "error": "No appropriate schema found for this type of content"
                }
            
            result["detected_type"] = detected_type
            return result
            
        except Exception as e:
            return {
                "success": False,
                "error": f"Auto-detection failed: {str(e)}"
            }

# Create validation agent
validation_agent = ValidationAgent()
print("Validation agent created successfully!")

Validation agent created successfully!


## Testing the Validation Agent

Let's test our validation agent with different types of content.

In [16]:
# Test 1: Task extraction
print("=== Test 1: Task Extraction ===")
task_text = """
We need to implement the new user authentication system by Friday, March 15th. 
This is a high priority task that should be assigned to the security team. 
It will probably take about 40 hours to complete and depends on the database 
migration being finished first.
"""

task_result = validation_agent.extract_tasks(task_text.strip())
print(f"Task extraction result:")
print(json.dumps(task_result, indent=2))
print()

=== Test 1: Task Extraction ===
🔄 Attempt 1/3 (temperature: 0.0)
✅ Success on attempt 1
Task extraction result:
{
  "success": true,
  "data": {
    "task": "Implement the new user authentication system",
    "priority": "high",
    "deadline": "2024-03-15",
    "assignee": null,
    "department": "security team",
    "estimated_hours": 40,
    "dependencies": [
      "database migration being finished"
    ]
  },
  "attempts": 1,
  "final_temperature": 0.0,
  "raw_response": "{\n  \"task\": \"Implement the new user authentication system\",\n  \"priority\": \"high\",\n  \"deadline\": \"2024-03-15\",\n  \"assignee\": null,\n  \"department\": \"security team\",\n  \"estimated_hours\": 40,\n  \"dependencies\": [\n    \"database migration being finished\"\n  ]\n}"
}



In [17]:
# Test 2: Person information extraction
print("=== Test 2: Person Information Extraction ===")
person_text = """
Sarah Johnson is a Senior Software Engineer at TechCorp with 8 years of experience 
in backend development. She specializes in Python, Go, and distributed systems. 
You can reach her at sarah.johnson@techcorp.com or call her at +1-555-0123.
"""

person_result = validation_agent.extract_person_info(person_text.strip())
print(f"Person extraction result:")
print(json.dumps(person_result, indent=2))
print()

=== Test 2: Person Information Extraction ===
🔄 Attempt 1/3 (temperature: 0.0)
✅ Success on attempt 1
Person extraction result:
{
  "success": true,
  "data": {
    "name": "Sarah Johnson",
    "role": "Senior Software Engineer",
    "company": "TechCorp",
    "email": "sarah.johnson@techcorp.com",
    "phone": "+1-555-0123",
    "skills": [
      "backend development",
      "Python",
      "Go",
      "distributed systems"
    ],
    "experience_years": 8
  },
  "attempts": 1,
  "final_temperature": 0.0,
  "raw_response": "{\n  \"name\": \"Sarah Johnson\",\n  \"role\": \"Senior Software Engineer\",\n  \"company\": \"TechCorp\",\n  \"email\": \"sarah.johnson@techcorp.com\",\n  \"phone\": \"+1-555-0123\",\n  \"skills\": [\n    \"backend development\",\n    \"Python\",\n    \"Go\",\n    \"distributed systems\"\n  ],\n  \"experience_years\": 8\n}"
}



In [18]:
# Test 3: Project plan extraction
print("=== Test 3: Project Plan Extraction ===")
project_text = """
Project Apollo: AI-Powered Customer Service Bot

We're launching an AI-powered customer service chatbot to improve response times 
and customer satisfaction. The main objectives are to reduce average response time 
by 50%, handle 80% of common queries automatically, and provide 24/7 support.

Timeline: 6 months from project start
Budget: $250,000
Team: We'll need 5-6 people including AI engineers, UX designers, and QA testers
Technologies: Python, Azure OpenAI, React, Node.js
Main risks: Model accuracy, integration complexity, user adoption
"""

project_result = validation_agent.extract_project_plan(project_text.strip())
print(f"Project extraction result:")
print(json.dumps(project_result, indent=2))
print()

=== Test 3: Project Plan Extraction ===
🔄 Attempt 1/3 (temperature: 0.0)
✅ Success on attempt 1
Project extraction result:
{
  "success": true,
  "data": {
    "project_name": "Project Apollo: AI-Powered Customer Service Bot",
    "description": "We're launching an AI-powered customer service chatbot to improve response times and customer satisfaction.",
    "objectives": [
      "Reduce average response time by 50%",
      "Handle 80% of common queries automatically",
      "Provide 24/7 support"
    ],
    "timeline": "6 months from project start",
    "budget": "$250,000",
    "team_size": 6,
    "technologies": [
      "Python",
      "Azure OpenAI",
      "React",
      "Node.js"
    ],
    "risks": [
      "Model accuracy",
      "Integration complexity",
      "User adoption"
    ]
  },
  "attempts": 1,
  "final_temperature": 0.0,
  "raw_response": "{\n  \"project_name\": \"Project Apollo: AI-Powered Customer Service Bot\",\n  \"description\": \"We're launching an AI-powered cus

In [19]:
# Test 4: Auto-detection
print("=== Test 4: Auto-Detection ===")
mixed_text = """
We need to update the website footer with new contact information. 
This should be done by next Tuesday and assigned to the web development team.
"""

auto_result = validation_agent.auto_detect_and_extract(mixed_text.strip())
print(f"Auto-detection result:")
print(json.dumps(auto_result, indent=2))
print()

=== Test 4: Auto-Detection ===
🔍 Detected information type: task
🔄 Attempt 1/3 (temperature: 0.0)
✅ Success on attempt 1
Auto-detection result:
{
  "success": true,
  "data": {
    "task": "Update the website footer with new contact information",
    "priority": "medium",
    "deadline": "2024-06-18",
    "assignee": null,
    "department": "web development team",
    "estimated_hours": null,
    "dependencies": []
  },
  "attempts": 1,
  "final_temperature": 0.0,
  "raw_response": "{\n  \"task\": \"Update the website footer with new contact information\",\n  \"priority\": \"medium\",\n  \"deadline\": \"2024-06-18\",\n  \"assignee\": null,\n  \"department\": \"web development team\",\n  \"estimated_hours\": null,\n  \"dependencies\": []\n}",
  "detected_type": "task"
}



## Summary

In this notebook, we've implemented two essential building blocks for Azure OpenAI agents:

### Tools (Function Calling)
- **Weather API Integration**: Complete implementation of external API calls
- **Function Schema Definition**: Proper OpenAI function calling schema setup
- **Tool Agent**: Intelligent agent that can decide when and how to use tools
- **Error Handling**: Robust error handling for API failures

### Validation (Structured Extraction)
- **Pydantic Schemas**: Well-defined data structures for different content types
- **Structured Intelligence**: Reliable extraction using Azure OpenAI's JSON mode
- **Retry Logic**: Progressive temperature adjustment for better extraction
- **Auto-Detection**: Automatic content type detection and appropriate schema selection

These components provide the foundation for building sophisticated AI agents that can:
- Interact with external systems safely and reliably
- Extract and validate structured data from unstructured text
- Handle errors gracefully with appropriate fallback strategies
- Maintain data integrity through proper validation

**Next Steps**: Combine these with control flow and recovery mechanisms to build production-ready AI agents.